# 3. Store the data in your DB (those that are available) which should be partially those concerning the content level

In [3]:
import pandas as pd
from sqlalchemy import create_engine, inspect, MetaData, Table, Column, Integer, String, DateTime, Text, Boolean, ForeignKey
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy.dialects.postgresql import insert as pg_insert
from datetime import datetime
from faker import Faker

fake = Faker()

# Database connection details
host = "localhost"
port = "5432"
database = "karim_database"
username = "postgres"
password = "Karim123*"

# Create the connection string
connection_string = f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}"

# Create an engine instance
engine = create_engine(connection_string)
metadata = MetaData(bind=engine)



# Load the CSV file into a DataFrame
df = pd.read_csv("C:/Users/Karim Matragi/Desktop/Project Helper/day_4/day_4/data/Lorena_Instagram(CSV).csv")

def parse_timestamp(timestamp):
    try:
        return datetime.strptime(timestamp, "%m/%d/%Y")
    except Exception:
        return datetime.now()  # Return the current timestamp if parsing fails

def insert_data(table, data):
    insert_stmt = pg_insert(table).values(data).on_conflict_do_nothing()
    try:
        engine.execute(insert_stmt)
    except SQLAlchemyError as e:
        print(f"Error inserting into {table.name}: {e}")

# Insert data into the users table
for index, row in df.iterrows():
    user_data = {
        'user_id': row.get('ID', None),
        'username': row.get('Source', None),
        'email': fake.email(),
        'timestamp': parse_timestamp(row.get('Post date', None)),
        'is_active': True
    }
    insert_data(users, user_data)

# Insert data into the media table first
for index, row in df.iterrows():
    media_url = row.get('URL 1', None)
    media_type = 'unknown'
    media_data = {
        'media_id': index + 1,
        'post_id': None,  # This will be updated later when post_id is available
        'url': media_url,
        'type': media_type,
        'timestamp': parse_timestamp(row.get('Post date', None))
    }
    insert_data(media, media_data)

# Insert data into the content table
for index, row in df.iterrows():
    content_data = {
        'content_id': index + 1,
        'text': row.get('Description', 'N/A'),  # Provide a default value if 'Description' is None
        'geostamp': row.get('Geolocation', 'N/A'),  # Provide a default value if 'Geolocation' is None
        'timestamp': parse_timestamp(row.get('Post date', None)),
        'media_id': index + 1  # Ensure this matches an existing media_id
    }
    insert_data(content, content_data)

# Insert data into the post table
for index, row in df.iterrows():
    user_id = row.get('ID', None)
    
    # Check if the user exists in the users table
    user_exists = engine.execute(f"SELECT EXISTS(SELECT 1 FROM users WHERE user_id = {user_id})").scalar()
    
    if user_exists:
        media_url = row.get('URL 1', None)
        media_type = 'image' if '.jpg' in media_url else 'video' if '.mp4' in media_url else 'unknown'
        post_data = {
            'post_id': index + 1,
            'user_id': user_id,
            'content_id': index + 1,  # Ensure this matches an existing content_id
            'timestamp': parse_timestamp(row.get('Post date', None)),
            'media_type': media_type,
            'media_url': media_url,
            'caption': row.get('Caption', 'N/A'),  # Provide a default value if 'Caption' is None
            'like_count': row.get('Likes', 0),  # Default to 0 if 'Likes' is None
            'comment_count': row.get('Comments', 0)  # Default to 0 if 'Comments' is None
        }
        insert_data(post, post_data)
    else:
        print(f"Skipping post insertion for user_id {user_id} at row {index}: user does not exist")

# Update media table with correct post_id
for index, row in df.iterrows():
    update_query = media.update().values(post_id=index + 1).where(media.c.media_id == index + 1)
    try:
        engine.execute(update_query)
    except SQLAlchemyError as e:
        print(f"Error updating media with media_id={index + 1}: {e}")

# Insert data into the comment table
for index, row in df.iterrows():
    comment_data = {
        'comment_id': index + 1,
        'post_id': index + 1,
        'user_id': row.get('ID', None),
        'text': row.get('Comment', 'N/A'),  # Provide a default value if 'Comment' is None
        'timestamp': parse_timestamp(row.get('Comment date', None))
    }
    insert_data(comment, comment_data)

# Insert data into the likes table
for index, row in df.iterrows():
    like_data = {
        'like_id': index + 1,
        'post_id': index + 1,
        'user_id': row.get('ID', None),
        'timestamp': parse_timestamp(row.get('Like date', None))
    }
    insert_data(likes, like_data)

# Insert data into the tags table
for index, row in df.iterrows():
    tag_data = {
        'tag_id': index + 1,
        'name': row.get('Tag', 'N/A')  # Provide a default value if 'Tag' is None
    }
    insert_data(tags, tag_data)

# Insert data into the collection table
for index, row in df.iterrows():
    collection_data = {
        'collection_id': index + 1,
        'harvesting_tech': fake.random_int(min=1, max=10),
        'time_window': row.get('Time Window', 'N/A'),  # Provide a default value if 'Time Window' is None
        'geo_window': row.get('Geo Window', 'N/A'),  # Provide a default value if 'Geo Window' is None
        'timestamp': parse_timestamp(row.get('Collection date', None))
    }
    insert_data(collection, collection_data)

# Insert data into the teams table
for index, row in df.iterrows():
    team_data = {
        'team_id': index + 1,
        'name': row.get('Team Name', 'N/A')  # Provide a default value if 'Team Name' is None
    }
    insert_data(teams, team_data)

# Insert data into the experiment table
for index, row in df.iterrows():
    experiment_data = {
        'experiment_id': index + 1,
        'team_id': fake.random_int(min=1, max=10),
        'topic': row.get('Experiment Topic', 'N/A'),  # Provide a default value if 'Experiment Topic' is None
        'period': parse_timestamp(row.get('Experiment Period', None))
    }
    insert_data(experiment, experiment_data)

# Insert data into the scientist table
for index, row in df.iterrows():
    scientist_data = {
        'scientist_id': index + 1,
        'name': row.get('Scientist Name', 'N/A'),  # Provide a default value if 'Scientist Name' is None
        'team_id': fake.random_int(min=1, max=10),
        'seniority': 'junior'  # Provide a default value for 'seniority'
    }
    insert_data(scientist, scientist_data)

# Insert data into the research_question table
for index, row in df.iterrows():
    research_question_data = {
        'rq_id': index + 1,
        'research_question': row.get('Research Question', 'N/A')  # Provide a default value if 'Research Question' is None
    }
    insert_data(research_question, research_question_data)

# Insert data into the business_rule table
for index, row in df.iterrows():
    business_rule_data = {
        'business_id': index + 1,
        'business_rule': row.get('Business Rule', 'N/A')  # Provide a default value if 'Business Rule' is None
    }
    insert_data(business_rule, business_rule_data)

# Insert data into the experiment_tag table
for index, row in df.iterrows():
    experiment_tag_data = {
        'tag_id': index + 1,
        'tagName': row.get('Experiment Tag', 'N/A'),  # Provide a default value if 'Experiment Tag' is None
        'content': row.get('Tag Content', 'N/A')  # Provide a default value if 'Tag Content' is None
    }
    insert_data(experiment_tag, experiment_tag_data)

NameError: name 'users' is not defined

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# Database connection details
host = "localhost"
port = "5432"
database = "karim_database"
username = "postgres"
password = "Karim123*"

# Create the connection string
connection_string = f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}"

# Create an engine instance
engine = create_engine(connection_string)

# Function to execute a query and display results
def display_query_results(query):
    df = pd.read_sql(query, engine)
    display(df)

# users content media post comment likes tags collection business_rule experiment_tag experiment teams scientist research_question

# Verify data in experiment_tag table
display_query_results("SELECT * FROM experiment_tag LIMIT 10;")

# Verify data in media table
display_query_results("SELECT * FROM media LIMIT 10;")

# Verify data in experiment table
display_query_results("SELECT * FROM experiment LIMIT 10;")

# Verify data in users table
display_query_results("SELECT * FROM users LIMIT 10;")

# Verify data in posts table
display_query_results("SELECT * FROM post LIMIT 10;")

# Verify data in content table
display_query_results("SELECT * FROM content LIMIT 10;")

# Verify data in comment table
display_query_results("SELECT * FROM comment LIMIT 10;")

# Verify data in like table
display_query_results("SELECT * FROM likes LIMIT 10;")

# Verify data in tags table
display_query_results("SELECT * FROM tags LIMIT 10;")

# Verify data in collection table
display_query_results("SELECT * FROM collection LIMIT 10;")

# Verify data in business_rule table
display_query_results("SELECT * FROM business_rule LIMIT 10;")

# Verify data in teams table
display_query_results("SELECT * FROM teams LIMIT 10;")

# Verify data in scientist table
display_query_results("SELECT * FROM scientist LIMIT 10;")

# Verify data in research_question table
display_query_results("SELECT * FROM research_question LIMIT 10;")